You are given a 0-indexed array heights of positive integers, where heights[i] represents the height of the ith building.

If a person is in building i, they can move to any other building j if and only if i < j and heights[i] < heights[j].

You are also given another array queries where queries[i] = [ai, bi]. On the ith query, Alice is in building ai while Bob is in building bi.

Return an array ans where ans[i] is the index of the leftmost building where Alice and Bob can meet on the ith query. If Alice and Bob cannot move to a common building on query i, set ans[i] to -1.



Example 1:

Input: heights = [6,4,8,5,2,7], queries = [[0,1],[0,3],[2,4],[3,4],[2,2]]
Output: [2,5,-1,5,2]
Explanation: In the first query, Alice and Bob can move to building 2 since heights[0] < heights[2] and heights[1] < heights[2]. 
In the second query, Alice and Bob can move to building 5 since heights[0] < heights[5] and heights[3] < heights[5]. 
In the third query, Alice cannot meet Bob since Alice cannot move to any other building.
In the fourth query, Alice and Bob can move to building 5 since heights[3] < heights[5] and heights[4] < heights[5].
In the fifth query, Alice and Bob are already in the same building.  
For ans[i] != -1, It can be shown that ans[i] is the leftmost building where Alice and Bob can meet.
For ans[i] == -1, It can be shown that there is no building where Alice and Bob can meet.
Example 2:

Input: heights = [5,3,8,2,6,1,4,6], queries = [[0,7],[3,5],[5,2],[3,0],[1,6]]
Output: [7,6,-1,4,6]
Explanation: In the first query, Alice can directly move to Bob's building since heights[0] < heights[7].
In the second query, Alice and Bob can move to building 6 since heights[3] < heights[6] and heights[5] < heights[6].
In the third query, Alice cannot meet Bob since Bob cannot move to any other building.
In the fourth query, Alice and Bob can move to building 4 since heights[3] < heights[4] and heights[0] < heights[4].
In the fifth query, Alice can directly move to Bob's building since heights[1] < heights[6].
For ans[i] != -1, It can be shown that ans[i] is the leftmost building where Alice and Bob can meet.
For ans[i] == -1, It can be shown that there is no building where Alice and Bob can meet.



Constraints:

1 <= heights.length <= 5 * 104
1 <= heights[i] <= 109
1 <= queries.length <= 5 * 104
queries[i] = [ai, bi]
0 <= ai, bi <= heights.length - 1

In [ ]:
# Time limit걸린 문제..
class Solution:
    def leftmostBuildingQueries(self, heights: List[int], queries: List[List[int]]) -> List[int]:
        ans = []
        # O(n^2 sol..)
        for i in range(len(queries)):
            # low & high are index
            high = max(queries[i])
            low = min(queries[i])

            if high == low or heights[high] > heights[low] :
                ans.append(high)
                continue
            else:
                cond = False
                for j in range(high+1,len(heights)):
                    if heights[j] > heights[low] and heights[j] > heights[high]:
                        cond = True
                        ans.append(j)
                        break
                if not cond :
                    ans.append(-1) 

        return ans

In [4]:
temp = [5,1,2,6,123]
temp=sorted(temp)
temp


[1, 2, 5, 6, 123]

In [ ]:
from bisect import bisect_left

class Solution:
    def leftmostBuildingQueries(self, heights: List[int], queries: List[List[int]]) -> List[int]:
        n = len(heights)
        st = [[0] * 20 for _ in range(n)]
        Log = [-1] * (n + 1)
        Log[0] = -1
        for i in range(1, n + 1):
            Log[i] = Log[i >> 1] + 1
        for i in range(n):
            st[i][0] = heights[i]
        for i in range(1, 20):
            for j in range(n):
                if j + (1 << i) <= n:
                    st[j][i] = max(st[j][i - 1], st[j + (1 << (i - 1))][i - 1])

        def Ask(l, r):
            k = Log[r - l + 1]
            return max(st[l][k], st[r - (1 << k) + 1][k])

        res = []
        for l, r in queries:
            if l > r:
                l, r = r, l
            if l == r:
                res.append(l)
                continue
            if heights[r] > heights[l]:
                res.append(r)
                continue
            max_height = max(heights[r], heights[l])
            left, right = r + 1, n
            while left < right:
                mid = (left + right) // 2
                if Ask(r + 1, mid) > max_height:
                    right = mid
                else:
                    left = mid + 1
            res.append(left if left != n else -1)
        return res

In [ ]:
#C++
class Solution {
public:
    int n, sparseTable[50010][20], logValues[50010];

    int queryMaximum(int left, int right) {
        int k = logValues[right - left + 1];
        return max(sparseTable[left][k], sparseTable[right - (1 << k) + 1][k]);
    }

    vector<int> leftmostBuildingQueries(vector<int>& heights, vector<vector<int>>& queries) {
        int n = heights.size();
        sparseTable[n][0] = 2e9;
        logValues[0] = -1;
        for (int i = 1; i <= n; i++) {
            logValues[i] = logValues[i >> 1] + 1;
        }
        for (int i = 0; i < n; i++) {
            sparseTable[i][0] = heights[i];
        }
        for (int i = 1; i < 20; i++) {
            for (int j = 0; j + (1 << i) - 1 <= n; j++) {
                sparseTable[j][i] = max(sparseTable[j][i - 1], sparseTable[j + (1 << (i - 1))][i - 1]);
            }
        }
        int numQueries = queries.size();
        vector<int> results(numQueries);
        for (int queryIndex = 0; queryIndex < numQueries; queryIndex++) {
            int left = queries[queryIndex][0], right = queries[queryIndex][1];
            if (left > right) swap(left, right);
            if (left == right) {
                results[queryIndex] = left;
                continue;
            }
            if (heights[right] > heights[left]) {
                results[queryIndex] = right;
                continue;
            }
            int maxHeight = max(heights[right], heights[left]);
            int low = right, high = n, mid;
            while (low < high) {
                mid = (low + high) >> 1;
                if (queryMaximum(right, mid) > maxHeight) {
                    high = mid;
                } else {
                    low = mid + 1;
                }
            }
            results[queryIndex] = (low == n) ? -1 : low;
        }
        return results;
    }
};

In [ ]:
#C
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int max(int a, int b) {
    return a > b ? a : b;
}

int* leftmostBuildingQueries(int* heights, int heightsSize, int** queries, int queriesSize, int* queriesColSize, int* returnSize) {
    int n = heightsSize;
    int st[n][20];
    int Log[n + 1];
    Log[0] = -1;

    for (int i = 1; i <= n; i++) {
        Log[i] = Log[i >> 1] + 1;
    }

    for (int i = 0; i < n; i++) {
        st[i][0] = heights[i];
    }

    for (int i = 1; i < 20; i++) {
        for (int j = 0; j + (1 << i) <= n; j++) {
            st[j][i] = max(st[j][i - 1], st[j + (1 << (i - 1))][i - 1]);
        }
    }

    int* res = (int*)malloc(queriesSize * sizeof(int));
    *returnSize = queriesSize;

    for (int i = 0; i < queriesSize; i++) {
        int l = queries[i][0], r = queries[i][1];
        if (l > r) {
            int temp = l;
            l = r;
            r = temp;
        }

        if (l == r) {
            res[i] = l;
            continue;
        }

        if (heights[r] > heights[l]) {
            res[i] = r;
            continue;
        }

        int maxHeight = max(heights[l], heights[r]);
        int left = r + 1, right = n, mid;

        while (left < right) {
            mid = (left + right) / 2;
            int k = Log[mid - r + 1];
            int maxInRange = max(st[r][k], st[mid - (1 << k) + 1][k]);

            if (maxInRange > maxHeight) {
                right = mid;
            } else {
                left = mid + 1;
            }
        }

        res[i] = (left == n) ? -1 : left;
    }

    return res;
}

In [8]:
#Python heap
class Solution:
    import heapq
    def leftmostBuildingQueries(self, heights, queries):
        n, q = len(heights), len(queries)
        result = [-1] * q
        deferred = [[] for _ in range(n)]
        pq = []

        for i in range(q):
            a, b = queries[i]
            if a > b:
                a, b = b, a
            if a == b or heights[a] < heights[b]:
                result[i] = b
            else:
                deferred[b].append((heights[a], i))

        for i in range(n):
            for query in deferred[i]:
                heapq.heappush(pq, query)
            while pq and pq[0][0] < heights[i]:
                result[pq[0][1]] = i
                heapq.heappop(pq)

        return result

In [9]:
import heapq

pq = []
heapq.heappush(pq,[1,6])
heapq.heappush(pq,[2,3])
heapq.heappush(pq,[2,4])
for x in pq:
    print(x)

[1, 6]
[2, 3]
[2, 4]


In [ ]:
import heapq

# 데이터 예제: (id, value)
data = [(1, 10), (2, 5), (3, 15), (4, 7)]

# 힙 생성 (두 번째 값 기준 최소 힙)
min_heap = []
for item in data:
    heapq.heappush(min_heap, (item[1], item))  # 두 번째 값을 기준으로 삽입

# 힙에서 최소값 꺼내기
while min_heap:
    priority, original = heapq.heappop(min_heap)
    print(original)  # 출력: (2, 5), (4, 7), (1, 10), (3, 15)


1. 두 번째 값 기준 최소 힙
구현 방법
튜플의 두 번째 값을 첫 번째 요소로 사용하여 heapq에 삽입합니다.

코드 예제
python
코드 복사
import heapq

# 데이터 예제: (id, value)
data = [(1, 10), (2, 5), (3, 15), (4, 7)]

# 힙 생성 (두 번째 값 기준 최소 힙)
min_heap = []
for item in data:
    heapq.heappush(min_heap, (item[1], item))  # 두 번째 값을 기준으로 삽입

# 힙에서 최소값 꺼내기
while min_heap:
    priority, original = heapq.heappop(min_heap)
    print(original)  # 출력: (2, 5), (4, 7), (1, 10), (3, 15)
출력
scss
코드 복사
(2, 5)
(4, 7)
(1, 10)
(3, 15)
2. 두 번째 값 기준 최대 힙
구현 방법
heapq는 최소 힙만 지원하므로, 최대 힙을 구현하려면 두 번째 값의 부호를 반전하여 삽입하면 됩니다.

코드 예제
python
코드 복사
import heapq

# 데이터 예제: (id, value)
data = [(1, 10), (2, 5), (3, 15), (4, 7)]

# 힙 생성 (두 번째 값 기준 최대 힙)
max_heap = []
for item in data:
    heapq.heappush(max_heap, (-item[1], item))  # 두 번째 값의 부호를 반전하여 삽입

# 힙에서 최대값 꺼내기
while max_heap:
    priority, original = heapq.heappop(max_heap)
    print(original)  # 출력: (3, 15), (1, 10), (4, 7), (2, 5)
출력
scss
코드 복사
(3, 15)
(1, 10)
(4, 7)
(2, 5)
3. 함수화된 힙 관리
두 번째 값 기준으로 최소/최대 힙을 만드는 작업을 함수로 만들어 반복적인 코드를 줄일 수 있습니다.

코드 예제
python
코드 복사
import heapq

def make_heap(data, key_index, max_heap=False):
    heap = []
    for item in data:
        priority = item[key_index]
        if max_heap:
            priority = -priority  # 최대 힙 구현
        heapq.heappush(heap, (priority, item))
    return heap

def pop_heap(heap, max_heap=False):
    priority, item = heapq.heappop(heap)
    if max_heap:
        priority = -priority  # 부호 복원
    return item

# 데이터 예제
data = [(1, 10), (2, 5), (3, 15), (4, 7)]

# 최소 힙
min_heap = make_heap(data, key_index=1, max_heap=False)
print([pop_heap(min_heap) for _ in range(len(min_heap))])
# 출력: [(2, 5), (4, 7), (1, 10), (3, 15)]

# 최대 힙
max_heap = make_heap(data, key_index=1, max_heap=True)
print([pop_heap(max_heap) for _ in range(len(max_heap))])
# 출력: [(3, 15), (1, 10), (4, 7), (2, 5)]
요약
최소 힙:

두 번째 값을 첫 번째 요소로 삽입: (item[1], item) 형태로 heappush.
최대 힙:

두 번째 값의 부호를 반전: (-item[1], item) 형태로 heappush.
함수화:

반복 코드를 줄이고 재사용성을 높이는 함수 make_heap 및 pop_heap 활용.
이 방식을 통해 heapq를 원하는 기준으로 손쉽게 커스터마이즈할 수 있습니다. 😊








# C++ heap
#include <vector>
#include <queue>
#include <utility>
using namespace std;

class Solution {
public:
    vector<int> leftmostBuildingQueries(vector<int>& heights, vector<vector<int>>& queries) {
        int n = heights.size();
        int q = queries.size();
        vector<int> result(q, -1);
        vector<vector<pair<int, int>>> deferred(n); // 각 빌딩별 처리해야 할 쿼리 저장
        priority_queue<pair<int, int>, vector<pair<int, int>>, greater<>> pq; // 최소 힙
        
        // 1. 쿼리 분류
        for (int i = 0; i < q; ++i) {
            int a = queries[i][0];
            int b = queries[i][1];
            if (a > b) swap(a, b); // 항상 a <= b 유지
            if (a == b || heights[a] < heights[b]) {
                result[i] = b; // 바로 만날 수 있는 경우
            } else {
                deferred[b].emplace_back(heights[a], i); // deferred[b]에 쿼리 저장
            }
        }
        
        // 2. 빌딩 순회
        for (int i = 0; i < n; ++i) {
            // 현재 빌딩에서 처리해야 할 쿼리를 힙에 추가
            for (auto& query : deferred[i]) {
                pq.push(query); // (Alice의 높이, 쿼리 인덱스)
            }
            
            // 힙에서 조건을 만족하는 쿼리를 처리
            while (!pq.empty() && pq.top().first < heights[i]) {
                result[pq.top().second] = i; // 현재 빌딩에서 Alice와 Bob이 만날 수 있음
                pq.pop();
            }
        }
        
        return result;
    }
};
